In [24]:
//default_exp nn

# nn

> A few modules that can be used to build neural nets.

In [1]:
import {shape,transpose,dotProduct,randn,zeros,matrixSum,matrixSubtract1d,matrixSubtract2d,matrixMultiply1d,matrixMultiply2d} from './src/util';
import {head,tail,readCsv,IRIS_CLASS_MAP,IrisRowHandler} from './src/data';

If you want to use this js module in an html page, you'll probably need something like;
```
<script type="module">
  import {shape,transpose,dotProduct,randn,zeros,matrixSum,matrixSubtract2d,matrixMultiply2d} from './src/util.js';
  import {Sigmoid, BinaryCrossEntropyLoss, Linear} from './src/nn.js'
</script>
```
See: demo.html

In [2]:
function testEq(expected,actual) {
    if (Array.isArray(expected)) {
        expected=JSON.stringify(expected);
        actual=JSON.stringify(actual);
    }
    if (expected!==actual) {
        throw Error(`Expected ${expected} but found ${actual}`);
    }
}

In [3]:
/**
*/
class BinaryCrossEntropyLoss {
    _forward1d(yPred1d,yTrue1d) {
        const temp=[];
        yPred1d.forEach(function (yPred, i) {
            let tempValue=yPred;
            tempValue=(yTrue1d[i]==1.) ? tempValue : 1-tempValue;
            tempValue=Math.log(tempValue);
            temp.push(tempValue);
        });
        return -temp.reduce((a,b) => a+b) / temp.length;
    }
    forward(yPred2d,yTrue2d) {
        this.yPred2d=yPred2d;
        this.yTrue2d=yTrue2d;
        const lossValue1d=yPred2d.map((yPred1d,i) => this._forward1d(yPred1d,yTrue2d[i]));
        return lossValue1d.reduce((a,b) => a+b) / lossValue1d.length;
    }
    _backward1d(yPred1d,yTrue1d) {
        const temp=[];
        yPred1d.forEach(function (yPred, i) { // TODO: rewrite with map
            let tempValue=(yTrue1d[i]==1.) ? -1/yPred : 1/(1-yPred);
            temp.push(tempValue);
        });
        return temp;
    }
    backward() {
        const yTrue2d=this.yTrue2d;
        this.grad=this.yPred2d.map((yPred1d,i) => this._backward1d(yPred1d,yTrue2d[i]));
        return this.grad;
    }
}

In [4]:
/**
*/
class Sigmoid {
    forward(x2d) {
        this.results=x2d.map(x1d => x1d.map(x => 1./(1.+Math.pow(Math.E, -x))));
        return this.results;
    }
    backward(gradients) {
        // `s * (1.-s)` calculates sigmoid grad, then we chain gradients passed in
        this.grad=this.results.map((result,i) => result.map((s,j) => s * (1.-s) * gradients[i][j]));
        return this.grad;
    }
}

In [5]:
/**
*/
class ReLU {
    forward(x2d) {
        this.gradMask=zeros(...shape(x2d));
        return x2d.map((x1d,rowIndex) => x1d.map((x,colIndex) => {
            if (x>0) {
                this.gradMask[rowIndex][colIndex]=1;
            }
            return Math.max(0,x)
        }));
    }
    matrixMultiply(a2d, b2d) {
        return a2d.map((a1d,rowIndex) => a1d.map((a,colIndex) => a*b2d[rowIndex][colIndex]));
    }
    backward(gradient) {
        return this.matrixMultiply(this.gradMask,gradient);
    }
}

In [6]:
let relu = new ReLU();
let data = [
  [ -0.3132450550822199, 0.06746248970796562, 0.7502210053477679 ],
  [ 0.32586239499711434, 0.276573231917191, 0.4718188033994297 ],
  [ 0.3375259522729109, -1.4738907605515226, -0.11109898767917284 ],
  [ -0.6095143988686595, 1.094470501593892, -0.4982351760328258 ],
  [ 0.28664244098736347, -0.35879217465991975, -0.754257906608068 ]
];
testEq(relu.forward(data),relu.backward(data));

In [7]:
/**
*/
class Linear {
    constructor(inputDim,numHidden=1) {
        this.inputDim=inputDim;
        this.numHidden=numHidden;
        // Kaiming Init
        this.weights=matrixMultiply2d(randn(inputDim,numHidden), Math.sqrt(2.0/inputDim));
        this.bias=zeros(numHidden)
    }
    forward(x) {
        this.x=x; // shape(bs,inputDim)
        return matrixSum(dotProduct(x,this.weights), this.bias);
    }
    backward(gradient) { // gradient shape(bs,numHidden)
        // weightsGradient/biasGradient need to be the same shape as weights/bias
        this.weightsGradient=dotProduct(transpose(this.x), gradient);
        // this.biasGradient=gradient.sum(axis=0)
        this.biasGradient=transpose(gradient).map(col => col.reduce((a,b) => a+b));
        this.xGradient=dotProduct(gradient,transpose(this.weights));
        return this.xGradient;
    }
    update(lr) {
        // gradient calculations in backward don't account for batch size, so we do it here
        lr=lr/this.x.length;
        this.weights=matrixSubtract2d(this.weights,matrixMultiply2d(this.weightsGradient,lr));
        this.bias=matrixSubtract1d(this.bias,matrixMultiply1d(this.biasGradient,lr));
    }
}

In [8]:
/**
*/
class Learner {
    constructor(model, lossFn, data) {
        this.model=model;
        this.lossFn=lossFn;
        this.x=data[0];
        this.y=data[1];
    }
    forward(x) {
        for (let i=0; i<this.model.length; i++) {
            x=this.model[i].forward(x);
        }
        return x;
    }
    backward(gradients) {
        for (let i=this.model.length-1; i>=0; i--) {
            gradients=this.model[i].backward(gradients);
        }
        return gradients;
    }
    step(lr) {
        this.model.forEach(m => {
            if (typeof m.update=='function') {
                m.update(lr);
            }
        });
    }
    fit(epochs, lr=0.1) {
        for (let epoch=0; epoch<epochs; epoch++) {
            const preds=this.forward(this.x);
            const lossValue=this.lossFn.forward(preds, this.y);
            console.log('epoch',epoch,'lossValue',lossValue);
            this.lossFn.backward();
            this.backward(this.lossFn.grad);
            this.step(lr);
        }
    }
}

In [9]:
let data=readCsv('data/iris.data', new IrisRowHandler()).result;
let lossFn=new BinaryCrossEntropyLoss();
let model=[new Linear(4,3), new Sigmoid()];
let learn=new Learner(model, lossFn, data);
learn.fit(25);

epoch 0 lossValue 0.8389957671088064
epoch 1 lossValue 0.8027136839732344
epoch 2 lossValue 0.7699246629988102
epoch 3 lossValue 0.7402485169525342
epoch 4 lossValue 0.7133327886077487
epoch 5 lossValue 0.6888581246012238
epoch 6 lossValue 0.6665398924073669
epoch 7 lossValue 0.6461273496955018
epoch 8 lossValue 0.6274013720112672
epoch 9 lossValue 0.6101714492170357
epoch 10 lossValue 0.5942724224943255
epoch 11 lossValue 0.5795612568552356
epoch 12 lossValue 0.5659140195740663
epoch 13 lossValue 0.5532231507930658
epoch 14 lossValue 0.5413950580868366
epoch 15 lossValue 0.5303480333637416
epoch 16 lossValue 0.5200104715199867
epoch 17 lossValue 0.5103193608430758
epoch 18 lossValue 0.5012190117799306
epoch 19 lossValue 0.4926599909165668
epoch 20 lossValue 0.4845982292426418
epoch 21 lossValue 0.47699427695634655
epoch 22 lossValue 0.4698126805645998
epoch 23 lossValue 0.46302146148028206
epoch 24 lossValue 0.4565916785151733


In [14]:
let model=[new Linear(4,100), new ReLU(), new Linear(100,3), new Sigmoid()];
let learn=new Learner(model, lossFn, data);
learn.fit(25);

epoch 0 lossValue 0.8848755299025032
epoch 1 lossValue 0.39032216624505556
epoch 2 lossValue 0.3425505325347123
epoch 3 lossValue 0.31775424463127694
epoch 4 lossValue 0.30007911594060277
epoch 5 lossValue 0.2864977249267318
epoch 6 lossValue 0.27562102463151805
epoch 7 lossValue 0.2666364380194296
epoch 8 lossValue 0.2590210637544141
epoch 9 lossValue 0.25243452329900823
epoch 10 lossValue 0.24663798383408836
epoch 11 lossValue 0.2414588258892636
epoch 12 lossValue 0.23677712310682797
epoch 13 lossValue 0.2325020725406714
epoch 14 lossValue 0.22856866266117398
epoch 15 lossValue 0.22492704140151795
epoch 16 lossValue 0.22153020711914007
epoch 17 lossValue 0.21834214050191378
epoch 18 lossValue 0.21534192841738858
epoch 19 lossValue 0.21250778445511892
epoch 20 lossValue 0.20982204730218834
epoch 21 lossValue 0.20726901662362984
epoch 22 lossValue 0.20483353241629312
epoch 23 lossValue 0.20250461757775032
epoch 24 lossValue 0.20027301407369352


In [20]:
tail(learn.forward(data[0]))

-10 [ 0.0014427057965869436, 0.07737050360759991, 0.939179016996877 ]
-9 [ 0.002665054151022015, 0.09736167457485931, 0.903248651916144 ]
-8 [ 0.01001240372301815, 0.3794685608430616, 0.6928703897968426 ]
-7 [ 0.00110879844859113, 0.06834045967723933, 0.9470350487134357 ]
-6 [ 0.0020512233415297403, 0.048731627079308275, 0.9533873125181563 ]
-5 [ 0.00244201444568896, 0.11482047537409476, 0.900382757639769 ]
-4 [ 0.0029226800052235278, 0.3672896850956114, 0.6200462621605491 ]
-3 [ 0.005662039971808958, 0.17511904458909797, 0.8570805027556981 ]
-2 [ 0.015602925419616196, 0.05897304084293871, 0.9009636136148318 ]
-1 [ 0.02436071893509965, 0.2767323766011408, 0.7999814217061862 ]


In [21]:
tail(data[1])

-10 [ 0, 0, 1 ]
-9 [ 0, 0, 1 ]
-8 [ 0, 0, 1 ]
-7 [ 0, 0, 1 ]
-6 [ 0, 0, 1 ]
-5 [ 0, 0, 1 ]
-4 [ 0, 0, 1 ]
-3 [ 0, 0, 1 ]
-2 [ 0, 0, 1 ]
-1 [ 0, 0, 1 ]


In [6]:
let data=readCsv('data/iris.data', new IrisRowHandler()).result;
let x=data[0],y=data[1];
console.log('shape(x)',shape(x));
console.log('shape(y)',shape(y));

shape(x) [ 150, 4 ]
shape(y) [ 150, 3 ]


In [10]:
let loss_fn=new BinaryCrossEntropyLoss()
let sig=new Sigmoid()
let lin=new Linear(4,3);
let y_pred=[-1,-1,-1];
for (let epoch = 0; epoch < 25; epoch++) {
    y_pred=sig.forward(lin.forward(x));
    let loss_value=loss_fn.forward(y_pred,y);
    console.log('epoch',epoch,'loss_value',loss_value);
    loss_fn.backward();
    sig.backward(loss_fn.grad);
    let xGradient=lin.backward(sig.grad);
    lin.update(.1);
}

epoch 0 loss_value 1.3664833195783415
epoch 1 loss_value 0.9230028634517524
epoch 2 loss_value 0.60032451870065
epoch 3 loss_value 0.5362998961895056
epoch 4 loss_value 0.5205330854081591
epoch 5 loss_value 0.5077603393177204
epoch 6 loss_value 0.49629636568616253
epoch 7 loss_value 0.4858011542125443
epoch 8 loss_value 0.4761480442265925
epoch 9 loss_value 0.4672510527142857
epoch 10 loss_value 0.4590371125516729
epoch 11 loss_value 0.45144103129724594
epoch 12 loss_value 0.44440418028041184
epoch 13 loss_value 0.43787384173198934
epoch 14 loss_value 0.43180265362408493
epoch 15 loss_value 0.42614809723817265
epoch 16 loss_value 0.42087201713834005
epoch 17 loss_value 0.4159401767884066
epoch 18 loss_value 0.4113218507626792
epoch 19 loss_value 0.4069894535556852
epoch 20 loss_value 0.4029182038424946
epoch 21 loss_value 0.3990858223524826
epoch 22 loss_value 0.395472261092022
epoch 23 loss_value 0.3920594614352566
epoch 24 loss_value 0.3888311385370368


In [8]:
head(y_pred)
tail(y_pred)

0 [ 0.8560821065896508, 0.3443693027417149, 0.10502875040392985 ]
1 [ 0.7648508214197969, 0.2880176308303962, 0.11437896295182959 ]
2 [ 0.8319491731478371, 0.35684972435393975, 0.12437493518096367 ]
3 [ 0.7897243252816688, 0.33935997586458366, 0.13957037836637315 ]
4 [ 0.8753800940612789, 0.37713051453263374, 0.11147209018781531 ]
5 [ 0.869643859862119, 0.4225384173187039, 0.12080369127103169 ]
6 [ 0.8559333187555294, 0.4385849480444964, 0.14949795509764038 ]
7 [ 0.8312334350373288, 0.3341946041645768, 0.11423644545313136 ]
8 [ 0.7700050585277568, 0.34032410462415397, 0.14864133678279845 ]
9 [ 0.7782984462534168, 0.2664709792168673, 0.10856130087092707 ]
-10 [ 0.026349578645452402, 0.5173198522717077, 0.6731372855594888 ]
-9 [ 0.03890290836737534, 0.4863879277118938, 0.5756986203863386 ]
-8 [ 0.036831897773210415, 0.43170598393451787, 0.6337931024130123 ]
-7 [ 0.023184146488218562, 0.4586275148984692, 0.6635331694761722 ]
-6 [ 0.030480514483700552, 0.5859491148392643, 0.705604102472983

In [ ]:
loss_fn=new BinaryCrossEntropyLoss()
sig=new Sigmoid()
lin=new Linear(10,4);
console.log('shape(lin.weights)',shape(lin.weights), 'shape(lin.bias)',shape(lin.bias));
x=[
    [1,0,0,0,0,0,0,0,0,0],
    [0,1,0,0,0,0,0,0,0,0],
    [0,0,1,0,0,0,0,0,0,0],
    [0,0,0,1,0,0,0,0,0,0],
    [0,0,0,0,1,0,0,0,0,0],
    [0,0,0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,1,0,0,0],
    [0,0,0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,0,0,1]
];
y=[
    [0,0,0,0],
    [1,0,0,0],
    [0,1,0,0],
    [1,1,0,0],
    [0,0,1,0],
    [1,0,1,0],
    [0,1,1,0],
    [1,1,1,0],
    [0,0,0,1],
    [1,0,0,1]
];

for (let index = 0; index < 1000; index++) {
    y_pred=sig.forward(lin.forward(x));
    loss_value=loss_fn.forward(y_pred,y);
    // console.log('loss_value',loss_value);
    loss_fn.backward();
    sig.backward(loss_fn.grad);
    xGradient=lin.backward(sig.grad);
    lin.update(.1);
}
console.log(y_pred)

In [11]:
// const csv = require('csv-parser');
const fs = require('fs');

fs.createReadStream('data/iris.data')
//   .pipe(csv())
  .on('data', (row) => {
    console.log(row);
  })
  .on('end', () => {
    console.log('CSV file successfully processed');
  });

ReadStream {
  _readableState: ReadableState {
    objectMode: false,
    highWaterMark: 65536,
    buffer: BufferList { head: null, tail: null, length: 0 },
    length: 0,
    pipes: [],
    flowing: true,
    ended: false,
    endEmitted: false,
    reading: false,
    sync: true,
    needReadable: false,
    emittedReadable: false,
    readableListening: false,
    resumeScheduled: true,
    errorEmitted: false,
    emitClose: true,
    autoDestroy: false,
    destroyed: false,
    errored: null,
    closed: false,
    closeEmitted: false,
    defaultEncoding: 'utf8',
    awaitDrainWriters: null,
    multiAwaitDrain: false,
    readingMore: false,
    decoder: null,
    encoding: null,
    [Symbol(kPaused)]: false
  },
  _events: [Object: null prototype] {
    end: [ [Function (anonymous)], [Function (anonymous)] ],
    data: [Function (anonymous)]
  },
  _eventsCount: 2,
  _maxListeners: undefined,
  path: 'data/iris.data',
  fd: null,
  flags: 'r',
  mode: 438,
  start: undefined,

In [18]:
var fs = require('fs');
let irisData='';
fs.readFile( 'data/iris.data', function (err, data) {
    if (err) {
        throw err; 
    }
//   console.log(data.toString());
    irisData=data.toString();
});

In [21]:
irisData.split('\n')

[
  '5.1,3.5,1.4,0.2,Iris-setosa',
  '4.9,3.0,1.4,0.2,Iris-setosa',
  '4.7,3.2,1.3,0.2,Iris-setosa',
  '4.6,3.1,1.5,0.2,Iris-setosa',
  '5.0,3.6,1.4,0.2,Iris-setosa',
  '5.4,3.9,1.7,0.4,Iris-setosa',
  '4.6,3.4,1.4,0.3,Iris-setosa',
  '5.0,3.4,1.5,0.2,Iris-setosa',
  '4.4,2.9,1.4,0.2,Iris-setosa',
  '4.9,3.1,1.5,0.1,Iris-setosa',
  '5.4,3.7,1.5,0.2,Iris-setosa',
  '4.8,3.4,1.6,0.2,Iris-setosa',
  '4.8,3.0,1.4,0.1,Iris-setosa',
  '4.3,3.0,1.1,0.1,Iris-setosa',
  '5.8,4.0,1.2,0.2,Iris-setosa',
  '5.7,4.4,1.5,0.4,Iris-setosa',
  '5.4,3.9,1.3,0.4,Iris-setosa',
  '5.1,3.5,1.4,0.3,Iris-setosa',
  '5.7,3.8,1.7,0.3,Iris-setosa',
  '5.1,3.8,1.5,0.3,Iris-setosa',
  '5.4,3.4,1.7,0.2,Iris-setosa',
  '5.1,3.7,1.5,0.4,Iris-setosa',
  '4.6,3.6,1.0,0.2,Iris-setosa',
  '5.1,3.3,1.7,0.5,Iris-setosa',
  '4.8,3.4,1.9,0.2,Iris-setosa',
  '5.0,3.0,1.6,0.2,Iris-setosa',
  '5.0,3.4,1.6,0.4,Iris-setosa',
  '5.2,3.5,1.5,0.2,Iris-setosa',
  '5.2,3.4,1.4,0.2,Iris-setosa',
  '4.7,3.2,1.6,0.2,Iris-setosa',
  '4.8,3

In [ ]:
export {Sigmoid, BinaryCrossEntropyLoss, Linear}